In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)
    
debug = False

print(ts_info_fullfn)

/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/09_ts_data/09_debris_tsinfo.nc


In [4]:
add_ts_fns = True
if add_ts_fns:
    
    # Load all debris-covered glaciers
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)
    
    dc_rgiids = [x.split('-')[1] for x in dc_shp.RGIId.values]
    print(len(dc_rgiids), 'ex:', dc_rgiids[0],'\n')
    main_glac_rgi_all = debris_prms.selectglaciersrgitable(dc_rgiids)

    # Ts filenames dataframe
    if os.path.exists(debris_prms.ts_fp + debris_prms.ts_fns_fn):
        ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    else:
        ts_fns_cns = ['RGIId', 'ts_fullfn', 'ts_year_fullfn', 'ts_doy_fullfn', 'ts_dayfrac_fullfn']
        ts_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_all.shape[0], len(ts_fns_cns))), columns=ts_fns_cns)
        ts_fns_df['RGIId'] = main_glac_rgi_all['RGIId']
        
    # ===== PROCESS EACH GLACIER =====
    for nglac, glac_idx in enumerate(main_glac_rgi_all.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_all.index.values[2579]]):
        glac_str = main_glac_rgi_all.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_all.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        if os.path.exists(thick_dir + thick_fn):
#         if os.path.exists(thick_dir + thick_fn) and '0.0' in ts_fns_df.loc[glac_idx,:].values:
#             print(nglac, glac_idx, rgiid)

            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # ==== CHECK IF TIF HAS SURFACE TEMPERATURE DATA OVER THE GLACIER =====
            ts_fullfns = []
            ts_year_fullfns = []
            ts_doy_fullfns = []
            ts_dayfrac_fullfns = []
            ts_fn_wglacier = None
            ts_year_fn_wglacier = None
            ts_doy_fn_wglacier = None
            ts_dayfrac_fn_wglacier = None
            for i in os.listdir(debris_prms.ts_fp):
                if i.endswith('.tif'):
                    if 'tsurfC' in i:
                        ts_fullfns.append(debris_prms.ts_fp + i)
                    elif 'year' in i:
                        ts_year_fullfns.append(debris_prms.ts_fp + i)
                    elif 'doy' in i:
                        ts_doy_fullfns.append(debris_prms.ts_fp + i)
                    elif 'dayfrac' in i:
                        ts_dayfrac_fullfns.append(debris_prms.ts_fp + i)
                        
            ts_fullfns = sorted(ts_fullfns)
            ts_year_fullfns = sorted(ts_year_fullfns)
            ts_doy_fullfns = sorted(ts_doy_fullfns)
            ts_dayfrac_fullfns = sorted(ts_dayfrac_fullfns)

            if debug:
                print(ts_fullfns)
                print(ts_year_fullfns)
                print(ts_doy_fullfns)
                print(ts_dayfrac_fullfns)

            # Add the filenames
            fn_dict = OrderedDict()
            # DEM
            z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
            z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
            fn_dict['z1'] = z1_fp + z1_fn
            # Ice thickness
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            fn_dict['ice_thick'] = thick_dir + thick_fn
            # Ts
            ts_dict_keys = []
            for nfn, ts_fn in enumerate(ts_fullfns):
                fn_dict['ts_' + str(nfn)] = ts_fn
                ts_dict_keys.append('ts_' + str(nfn))
            ts_year_dict_keys = []
            for nfn, ts_year_fn in enumerate(ts_year_fullfns):
                fn_dict['ts_year_' + str(nfn)] = ts_year_fn
                ts_year_dict_keys.append('ts_year_' + str(nfn))
            ts_doy_dict_keys = []
            for nfn, ts_doy_fn in enumerate(ts_doy_fullfns):
                fn_dict['ts_doy_' + str(nfn)] = ts_doy_fn
                ts_doy_dict_keys.append('ts_doy_' + str(nfn))
            ts_dayfrac_dict_keys = []  
            for nfn, ts_dayfrac_fn in enumerate(ts_dayfrac_fullfns):
                fn_dict['ts_dayfrac_' + str(nfn)] = ts_dayfrac_fn
                ts_dayfrac_dict_keys.append('ts_dayfrac_' + str(nfn))

            # ===== PROCESS THE DATA =====
            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
            #Warp everything to common res/extent/proj
            z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
            z1_res = np.min([z1_gt[1], -z1_gt[5]])
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                               t_srs=gf.aea_srs, verbose=False, r='cubic')
            ds_dict = dict(zip(fn_dict.keys(), ds_list))
            gf.ds_dict = ds_dict

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                glac_geom_mask_copy = glac_geom_mask.copy()
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                # Debris cover
                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)

                # ====== SURFACE TEMPERATURE =====
                find_ts = True
                ts_key_count = 0
                if debug:
                    print('\nSURFACE TEMPERATURE')
                while find_ts and ts_key_count < len(ts_dict_keys):
                    ts_key = ts_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        ts_fn_wglacier = ts_fullfns[ts_key_count]
                        find_ts = False
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_fn:', ts_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = malib.calcperc(var_full2plot, (2,98))
                            plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'Ts (degC)', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS YEAR =====
                find_ts_year = True
                ts_key_count = 0
                if debug:
                    print('\nYEAR')
                while find_ts_year and ts_key_count < len(ts_year_dict_keys):
                    ts_key = ts_year_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_year_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        ts_year_fn_wglacier = ts_year_fullfns[ts_key_count]
                        find_ts_year = False
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_year_fn:', ts_year_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (2013,2018)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_year'], 'inferno', 'Year', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DOY =====
                find_ts_doy = True
                ts_key_count = 0
                if debug:
                    print('\nDAY OF YEAR')
                while find_ts_doy and ts_key_count < len(ts_doy_dict_keys):
                    ts_key = ts_doy_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_doy_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        ts_doy_fn_wglacier = ts_doy_fullfns[ts_key_count]
                        find_ts_doy = False
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_doy_fn:', ts_doy_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (160,280)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_doy'], 'inferno', 'DOY', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DAYFRAC =====
                find_ts_dayfrac = True
                ts_key_count = 0
                if debug:
                    print('\nDAY FRACTION')
                while find_ts_dayfrac and ts_key_count < len(ts_dayfrac_dict_keys):
                    ts_key = ts_dayfrac_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_dayfrac_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        ts_dayfrac_fn_wglacier = ts_dayfrac_fullfns[ts_key_count]
                        find_ts_dayfrac = False
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_dayfrac_fn:', ts_dayfrac_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (0,24)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_dayfrac'], 'inferno', 'dayfrac', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                
                        
            # ===== Add layers =====
            # Record filenames if all exist
            if (ts_fn_wglacier is not None and ts_year_fn_wglacier is not None and ts_doy_fn_wglacier is not None and 
                ts_dayfrac_fn_wglacier is not None):                    
                ts_fns_df.loc[glac_idx,:] = [rgiid, ts_fn_wglacier, ts_year_fn_wglacier, ts_doy_fn_wglacier, 
                                             ts_dayfrac_fn_wglacier]
                    
    # Save updated filenames
    ts_fns_df.to_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn, index=False)
        


527 ex: 09.00014 

527 glaciers in region 9 are included in this model run: ['00014', '00016', '00025', '00027', '00028', '00029', '00031', '00033', '00034', '00035', '00038', '00040', '00051', '00053', '00055', '00056', '00057', '00058', '00060', '00061', '00062', '00064', '00065', '00066', '00067', '00069', '00070', '00071', '00072', '00073', '00077', '00079', '00080', '00081', '00083', '00086', '00087', '00088', '00090', '00091', '00092', '00093', '00094', '00095', '00096', '00097', '00099', '00101', '00102', '00103'] and more
This study is focusing on 527 glaciers in region [9]
0 0 RGI60-09.00014
1 1 RGI60-09.00016


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


2 2 RGI60-09.00025
3 3 RGI60-09.00027
4 4 RGI60-09.00028
5 5 RGI60-09.00029
6 6 RGI60-09.00031
7 7 RGI60-09.00033
8 8 RGI60-09.00034
9 9 RGI60-09.00035
10 10 RGI60-09.00038
11 11 RGI60-09.00040
12 12 RGI60-09.00051
13 13 RGI60-09.00053
14 14 RGI60-09.00055
15 15 RGI60-09.00056
16 16 RGI60-09.00057
17 17 RGI60-09.00058
18 18 RGI60-09.00060
19 19 RGI60-09.00061
20 20 RGI60-09.00062
21 21 RGI60-09.00064
22 22 RGI60-09.00065
23 23 RGI60-09.00066
24 24 RGI60-09.00067
25 25 RGI60-09.00069
26 26 RGI60-09.00070
27 27 RGI60-09.00071
28 28 RGI60-09.00072
29 29 RGI60-09.00073
30 30 RGI60-09.00077
31 31 RGI60-09.00079
32 32 RGI60-09.00080
33 33 RGI60-09.00081
34 34 RGI60-09.00083
35 35 RGI60-09.00086
36 36 RGI60-09.00087
37 37 RGI60-09.00088
38 38 RGI60-09.00090
39 39 RGI60-09.00091
40 40 RGI60-09.00092
41 41 RGI60-09.00093
42 42 RGI60-09.00094
43 43 RGI60-09.00095
44 44 RGI60-09.00096
45 45 RGI60-09.00097
46 46 RGI60-09.00099
47 47 RGI60-09.00101
48 48 RGI60-09.00102
49 49 RGI60-09.00103
50 50 RG

368 368 RGI60-09.00701
369 369 RGI60-09.00707
370 370 RGI60-09.00709
371 371 RGI60-09.00714
372 372 RGI60-09.00715
373 373 RGI60-09.00717
374 374 RGI60-09.00719
375 375 RGI60-09.00721
376 376 RGI60-09.00723
377 377 RGI60-09.00724
378 378 RGI60-09.00726
379 379 RGI60-09.00727
380 380 RGI60-09.00730
381 381 RGI60-09.00734
382 382 RGI60-09.00736
383 383 RGI60-09.00737
384 384 RGI60-09.00744
385 385 RGI60-09.00745
386 386 RGI60-09.00746
387 387 RGI60-09.00750
388 388 RGI60-09.00751
389 389 RGI60-09.00753
390 390 RGI60-09.00759
391 391 RGI60-09.00761
392 392 RGI60-09.00763
393 393 RGI60-09.00765
394 394 RGI60-09.00768
395 395 RGI60-09.00771
396 396 RGI60-09.00774
397 397 RGI60-09.00775
398 398 RGI60-09.00778
399 399 RGI60-09.00780
400 400 RGI60-09.00783
401 401 RGI60-09.00784
402 402 RGI60-09.00787
403 403 RGI60-09.00788
404 404 RGI60-09.00790
405 405 RGI60-09.00796
406 406 RGI60-09.00798
407 407 RGI60-09.00802
408 408 RGI60-09.00805
409 409 RGI60-09.00806
410 410 RGI60-09.00809
411 411 RGI

In [5]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        

# rgiid_list = ['18.00718']
# rgiid_fn_list = ['18.00718_mb_bins.csv']

rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)



main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

69 glaciers in region 9 are included in this model run: ['00014', '00033', '00051', '00053', '00055', '00056', '00060', '00071', '00077', '00086', '00120', '00123', '00129', '00132', '00134', '00138', '00153', '00197', '00201', '00207', '00215', '00230', '00244', '00246', '00248', '00254', '00255', '00275', '00277', '00302', '00303', '00307', '00312', '00329', '00343', '00352', '00359', '00362', '00367', '00369', '00392', '00393', '00394', '00441', '00478', '00484', '00524', '00551', '00552', '00572'] and more
This study is focusing on 69 glaciers in region [9]


In [6]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,13,RGI60-09.00014,57.4136,73.9902,9,2,2.687,165,469,391,...,0,9,20019999,14,09.00014,9.00014,57.4136,9.00014_mb_bins.csv,11.15,198156
1,32,RGI60-09.00033,57.9265,75.4014,9,2,4.296,160,358,250,...,0,9,20049999,33,09.00033,9.00033,57.9265,9.00033_mb_bins.csv,5.82,241927
2,50,RGI60-09.00051,57.3405,74.9907,9,2,437.891,91,1029,650,...,1,2,20049999,51,09.00051,9.00051,57.3405,9.00051_mb_bins.csv,0.42,1624900
3,52,RGI60-09.00053,57.1551,74.7632,9,2,164.901,7,1056,474,...,0,3,20049999,53,09.00053,9.00053,57.1551,9.00053_mb_bins.csv,0.93,1324615
4,54,RGI60-09.00055,65.3160,76.0499,9,2,383.049,64,781,561,...,1,9,20049999,55,09.00055,9.00055,65.3160,9.00055_mb_bins.csv,0.68,2389193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,922,RGI60-09.00923,57.1113,74.9614,9,2,87.873,63,942,347,...,0,9,20049999,923,09.00923,9.00923,57.1113,9.00923_mb_bins.csv,2.75,2278699
65,923,RGI60-09.00924,58.1297,75.1612,9,2,292.551,97,1048,620,...,0,2,20049999,924,09.00924,9.00924,58.1297,9.00924_mb_bins.csv,1.27,3183316
66,984,RGI60-09.00985,63.8595,80.8879,9,1,279.132,6,504,251,...,0,9,20010619,985,09.00985,9.00985,63.8595,9.00985_mb_bins.csv,0.53,1245357


In [7]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,13,RGI60-09.00014,57.4136,73.9902,9,2,2.687,165,469,391,...,14,09.00014,9.00014,57.4136,9.00014_mb_bins.csv,11.15,198156,74.25,56.75,7425N-5675E-debris_melt_curve.nc
1,32,RGI60-09.00033,57.9265,75.4014,9,2,4.296,160,358,250,...,33,09.00033,9.00033,57.9265,9.00033_mb_bins.csv,5.82,241927,75.00,57.75,7500N-5775E-debris_melt_curve.nc
2,50,RGI60-09.00051,57.3405,74.9907,9,2,437.891,91,1029,650,...,51,09.00051,9.00051,57.3405,9.00051_mb_bins.csv,0.42,1624900,75.00,57.75,7500N-5775E-debris_melt_curve.nc
3,52,RGI60-09.00053,57.1551,74.7632,9,2,164.901,7,1056,474,...,53,09.00053,9.00053,57.1551,9.00053_mb_bins.csv,0.93,1324615,74.75,56.75,7475N-5675E-debris_melt_curve.nc
4,54,RGI60-09.00055,65.3160,76.0499,9,2,383.049,64,781,561,...,55,09.00055,9.00055,65.3160,9.00055_mb_bins.csv,0.68,2389193,75.00,57.75,7500N-5775E-debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,922,RGI60-09.00923,57.1113,74.9614,9,2,87.873,63,942,347,...,923,09.00923,9.00923,57.1113,9.00923_mb_bins.csv,2.75,2278699,74.75,56.75,7475N-5675E-debris_melt_curve.nc
65,923,RGI60-09.00924,58.1297,75.1612,9,2,292.551,97,1048,620,...,924,09.00924,9.00924,58.1297,9.00924_mb_bins.csv,1.27,3183316,75.00,57.75,7500N-5775E-debris_melt_curve.nc
66,984,RGI60-09.00985,63.8595,80.8879,9,1,279.132,6,504,251,...,985,09.00985,9.00985,63.8595,9.00985_mb_bins.csv,0.53,1245357,80.00,58.50,8000N-5850E-debris_melt_curve.nc


In [8]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))


ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[24]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    lat_deg = lat_values[lat_idx]
    lon_deg = lon_values[lon_idx]
    
    print(nlatlon, lat_deg, lon_deg, ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[1]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid)

        # Ts filenames
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']
        ts_year_fn = ts_fns_df.loc[ts_fn_idx,'ts_year_fullfn']
        ts_doy_fn = ts_fns_df.loc[ts_fn_idx,'ts_doy_fullfn']
        ts_dayfrac_fn = ts_fns_df.loc[ts_fn_idx,'ts_dayfrac_fullfn']

        
        # Process debris-covered glaciers
        if (((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
             (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
            and '0.0' not in ts_fns_df.loc[ts_fn_idx].values):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()
            
            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, 
                          gf_add_ts=True, ts_fn=ts_fn,
                          gf_add_ts_info=True, ts_year_fn=ts_year_fn, ts_doy_fn=ts_doy_fn, ts_dayfrac_fn=ts_dayfrac_fn)

            # Isolate the debris-covered areas
            gf.ts = np.ma.array(gf.ts.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts.data)))
            gf.ts_dayfrac = np.ma.array(gf.ts_dayfrac.data, 
                                        mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_dayfrac.data)))
            gf.ts_year = np.ma.array(gf.ts_year.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_year.data)))
            gf.ts_doy = np.ma.array(gf.ts_doy.data, mask=np.ma.mask_or(gf.dc_mask, np.isnan(gf.ts_doy.data)))

#             # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
#         # ========================================================================================================
                    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))

    
    if len(doy_list) > 0 and len(year_list) > 0 and len(dayfrac_list) > 0:
        # Year
        year_mean_latlon = np.mean(year_list)
        year_std_latlon = np.std(year_list)
        year_med_latlon = malib.fast_median(year_list)
        year_mad_latlon = malib.mad(year_list)
        # Day of year
        #  adjust day of year to avoid issues with comparing JD 1 (Jan 1) and JD 365 (Dec 31),
        #  since the mean would be JD 182 (June), but should be JD 0.5 (Jan 1); also would affect std, etc.
        doy_array = np.array(doy_list)
        doy_array[doy_array > 365] = 365
        doy_array[doy_array > 182] = doy_array[doy_array > 182] - 365
        doy_mean_latlon = np.mean(doy_array)
        if doy_mean_latlon < 0:
            doy_mean_latlon += 365
        doy_std_latlon = np.std(doy_array)
        doy_med_latlon = malib.fast_median(doy_array)
        if doy_med_latlon < 0:
            doy_med_latlon += 365
        doy_mad_latlon = malib.mad(doy_array)
        dayfrac_mean_latlon = np.mean(dayfrac_list)
        dayfrac_std_latlon = np.std(dayfrac_list)
        dayfrac_med_latlon = malib.fast_median(dayfrac_list)
        dayfrac_mad_latlon = malib.mad(dayfrac_list)

        # Update array
        year_mean[lat_idx,lon_idx] = year_mean_latlon
        year_std[lat_idx,lon_idx] = year_std_latlon
        year_med[lat_idx,lon_idx] = year_med_latlon
        year_mad[lat_idx,lon_idx] = year_mad_latlon
        doy_mean[lat_idx,lon_idx] = doy_mean_latlon
        doy_std[lat_idx,lon_idx] = doy_std_latlon
        doy_med[lat_idx,lon_idx] = doy_med_latlon
        doy_mad[lat_idx,lon_idx] = doy_mad_latlon
        dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
        dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
        dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
        dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon

        print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
        print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
        print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
        print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 73.0 55.0 7300N-5500E-debris_melt_curve.nc
  year mean +/- std: 2014.6 1.5
  doy mean +/- std: 203.1 12.0
    doy median +/- mad: 197.8 21.6
  dayfrac mean +/- std: 8.389 0.103
1 73.25 55.25 7325N-5525E-debris_melt_curve.nc
  year mean +/- std: 2015.0 1.3
  doy mean +/- std: 196.4 13.0
    doy median +/- mad: 186.8 6.7
  dayfrac mean +/- std: 8.492 0.127
2 73.5 54.5 7350N-5450E-debris_melt_curve.nc
  year mean +/- std: 2015.7 0.9
  doy mean +/- std: 196.2 13.9
    doy median +/- mad: 186.8 0.0
  dayfrac mean +/- std: 8.541 0.092
3 73.5 55.0 7350N-5500E-debris_melt_curve.nc
  year mean +/- std: 2015.6 1.0
  doy mean +/- std: 193.1 9.0
    doy median +/- mad: 186.8 0.0
  dayfrac mean +/- std: 8.504 0.132
4 73.5 55.5 7350N-5550E-debris_melt_curve.nc
  year mean +/- std: 2015.1 1.3
  doy mean +/- std: 196.0 13.3
    doy median +/- mad: 186.8 0.0
  dayfrac mean +/- std: 8.502 0.124
5 73.5 56.0 7350N-5600E-debris_melt_curve.nc
  year mean +/- std: 2015.8 0.5
  doy mean +/- std: 203.1 16.1


In [9]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 31, longitude: 223)
Coordinates:
  * latitude      (latitude) float64 73.0 73.25 73.5 73.75 ... 80.0 80.25 80.5
  * longitude     (longitude) float64 49.5 49.75 50.0 ... 104.5 104.8 105.0
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude, l

In [10]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [11]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/09_ts_data/09_debris_tsinfo.nc'